In [1]:
# importing dataframe.xlsx file

import pandas as pd
df = pd.read_excel("dataframe_gerabstract.xlsx")
df.head()

,author,title,affiliation,email,abstract,year,journal,doi,street,city,zip
0,"Michael L. Brodie, Michael Stonebraker",DARWIN: On the Incremental Migration of Legacy...,Hochschule Albstadt-Sigmaringen,michael_l._brodie@hochschule_albstadt_sigmarin...,befragungen von mitarbeitern der verwaltung ha...,1997,ZA-Information / Zentralarchiv für Empirische ...,https://doi.org/10.1080/00036840701736115,Schoenebergerstrasse 79,Stralsund,18412
1,Frank Manola,Object Model Capabilities For Distributed Obje...,Fachhochschule für Verwaltung und Dienstleistu...,frank_manola@fachhochschule_für_verwaltung_und...,Acht Jahre nach der telefonischen Befragung vo...,2009,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Oldesloer Strasse 32,Suhl,98501
2,Frank Manola,Integrating Object-Oriented Applications and M...,Ostbayerische Technische Hochschule Amberg-Weiden,frank_manola@ostbayerische_technische_hochschu...,Das International Social Survey Programme (ISS...,2009,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Albrechtstrasse 84,Nußdorf,83131
3,"Frank Manola, Sandra Heiler",A RISC Object Model for Object System Interope...,Technische Hochschule Aschaffenburg,frank_manola@technische_hochschule_aschaffenbu...,Das GESIS-IZ Sozialwissenschaften hat im Auftr...,2008,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Gruenauer Strasse 59,Buxtehude,21614
4,Frank Manola,MetaObject Protocol Concepts for a RISC Object...,Fachhochschule Polizei Sachsen-Anhalt (Aschers...,frank_manola@fachhochschule_polizei_sachsen_an...,Die Studie präsentiert die Ergebnisse einer Be...,2005,ZUMA Nachrichten,https://doi.org/10.1080/00036840701736115,Prager Str 23,Weßling,82231


##### Generate annotation JSONs 

In [6]:
# run this file in the same folder where images and pdfs are placed
# change the file names to the templates you are working on, like here template is 10939, so just replace 10939

number=0
while True:
    
    names='10939_'+str(number)
    # change the number limit to number of files you are generating
    if number==2:
        break
        
    import pdfquery
    # loading a pdf
    pdf = pdfquery.PDFQuery(names+".pdf")
    pdf.load()
    # if you want, you can store the entire pdf as an xml file
    pdf.tree.write(names+".xml", pretty_print=True, encoding="utf-8")
    import xml.etree.ElementTree as ET
    tree = ET.parse(names+'.xml')
    root = tree.getroot()
   
    author=df['author'][number]
    title=df['title'][number]
    journal=df['journal'][number]  
   
    # create lists for all types of metadata bounding boxes 
    bounding_box_list=[]
    author_list=[]
    title_list=[]
    journal_list=[]

    # parse xml 
    for child in root:
        print('START')
        print (child.tag,"----", child.attrib)       
        for fchild in child:
            print (fchild.tag,"----", fchild.attrib)
            for ffchild in fchild:
                print (ffchild.tag,"----", ffchild.attrib)
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(ffchild.text)
                #print(ffchild.attrib['x0'])
                
                try:
                    bounding_box_list.append([ffchild.text.strip(),[ffchild.attrib['x0'],ffchild.attrib['x1'],ffchild.attrib['y0'],
                                                                    ffchild.attrib['y1']]])

                    if ffchild.text.strip() in author and ffchild.text.strip() is not '' and len(ffchild.text.strip())>1:
                        author_list.append([ffchild.text.strip(),[ffchild.attrib['x0'],ffchild.attrib['x1'],ffchild.attrib['y0'],
                                                                  ffchild.attrib['y1']]])
                    elif ffchild.text.strip() in title and ffchild.text.strip() is not '' and len(ffchild.text.strip())>1:
                        title_list.append([ffchild.text.strip(),[ffchild.attrib['x0'],ffchild.attrib['x1'],ffchild.attrib['y0'],
                                                                 ffchild.attrib['y1']]])
                    elif journal in ffchild.text.strip() and ffchild.text.strip() is not '' and len(ffchild.text.strip())>1:
                        journal_list.append([ffchild.text.strip(),[ffchild.attrib['x0'],ffchild.attrib['x1'],ffchild.attrib['y0'],
                                                                   ffchild.attrib['y1']]])
                except:
                    number=number+1
                    continue

                print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
            print('END')
            print('\n')

    coordinates_author = []
    coordinates_journal = []
    coordinates_title = []

    lst1=[]
    lst2=[]
    lst3=[]
    lst4=[]
    for lst in author_list:
        bboxes=lst[1]
        print((bboxes))
        lst1.append([float(bboxes[0])])
        lst2.append([float(bboxes[1])])
        lst3.append([842-float(bboxes[2])])
        lst4.append([842-float(bboxes[3])])  
    apt1=[min(lst1)[0],min(lst4)[0]]
    apt2=[min(lst1)[0],max(lst3)[0]]
    apt3=[max(lst2)[0],max(lst3)[0]]
    apt4=[max(lst2)[0],min(lst4)[0]]
    coordinates_author.append([ apt1,apt2,apt3,apt4 ])  
    print('-----------')
    print(coordinates_author)

    tlst1=[]
    tlst2=[]
    tlst3=[]
    tlst4=[]
    for lst in title_list:
        bboxes=lst[1]
        print((bboxes))
        tlst1.append([float(bboxes[0])])
        tlst2.append([float(bboxes[1])])
        tlst3.append([842-float(bboxes[2])])
        tlst4.append([842-float(bboxes[3])])  
    tpt1=[min(tlst1)[0],min(tlst4)[0]]
    tpt2=[min(tlst1)[0],max(tlst3)[0]]
    tpt3=[max(tlst2)[0],max(tlst3)[0]]
    tpt4=[max(tlst2)[0],min(tlst4)[0]]
    coordinates_title.append([ tpt1,tpt2,tpt3,tpt4 ])  
    print('-----------')
    print(coordinates_title)

    jlst1=[]
    jlst2=[]
    jlst3=[]
    jlst4=[]
    for lst in journal_list:
        bboxes=lst[1]
        print((bboxes))
        jlst1.append([float(bboxes[0])])
        jlst2.append([float(bboxes[1])])
        jlst3.append([842-float(bboxes[2])])
        jlst4.append([842-float(bboxes[3])])  
    jpt1=[min(jlst1)[0]-10,min(jlst4)[0]]
    jpt2=[min(jlst1)[0]-10,max(jlst3)[0]]
    jpt3=[max(jlst2)[0],max(jlst3)[0]]
    jpt4=[max(jlst2)[0],min(jlst4)[0]]
    coordinates_journal.append([ jpt1,jpt2,jpt3,jpt4 ])  
    print('-----------')
    print(coordinates_journal)

    lst=[coordinates_author,coordinates_title,coordinates_journal]
    lst
  
    # create a list of all metadata bounding boxes and use in JSON as 'shapes' value
    # abase --> for authors, tbase --> for titles and so on

    abase = {
          "label": "author",
          "points": [       [250,269],[250,283],[329,283],[329,269]         ],
          "shape_type": "polygon", "flags": {}
        }
    a1base={
          "label": "author",
          "points": [       [250,269],[250,283],[329,283],[329,269]         ],
          "shape_type": "polygon", "flags": {}
        }
    a2base={
          "label": "author",
          "points": [       [250,269],[250,283],[329,283],[329,269]         ],
          "shape_type": "polygon", "flags": {}
        }
    a3base={
          "label": "author",
          "points": [       [250,269],[250,283],[329,283],[329,269]         ],
          "shape_type": "polygon", "flags": {}
        }
    a4base={
          "label": "author",
          "points": [       [250,269],[250,283],[329,283],[329,269]         ],
          "shape_type": "polygon", "flags": {}
        }
    tbase = {
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t1base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t2base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t3base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t4base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t5base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t6base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t7base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    t8base={
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
           "shape_type": "polygon","flags": {}
        }
    jbase = {
          "label": "journal",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    j1base = {
          "label": "journal",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    j2base = {
      "label": "journal",
      "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
      "shape_type": "polygon","flags": {}
    }
    j3base = {
      "label": "journal",
      "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
      "shape_type": "polygon","flags": {}
    }
    dbase = {
          "label": "doi",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    d1base = {
          "label": "doi",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    dabase = {
          "label": "date",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    da1base = {
          "label": "date",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    absbase = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs1base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs2base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs3base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs4base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs5base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs6base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs7base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs8base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs9base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs10base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs11base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs12base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs13base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs14base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs15base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs16base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs17base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs18base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs19base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs20base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    abs21base = {
          "label": "abstract",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    adbase = {
          "label": "address",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    ad1base = {
          "label": "address",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    afbase = {
          "label": "affiliation",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    af1base = {
          "label": "affiliation",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    ebase = {
          "label": "email",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
    e1base = {
          "label": "email",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "shape_type": "polygon","flags": {}
        }
     
    ab=[abase,a1base,a2base,a3base,a4base]
    tb=[tbase,t1base,t2base,t3base,t4base,t5base,t6base,t7base,t8base]
    jb=[jbase,j1base,j2base,j3base]
    db=[dbase,d1base]
    dab=[dabase,da1base]
    absb=[absbase,abs1base,abs2base,abs3base,abs4base,abs5base,abs6base,abs7base,abs8base,abs9base,abs10base,abs11base,abs12base,abs13base,abs14base,abs15base,abs16base,abs17base,abs18base,abs19base,abs20base,abs21base]
    adb=[adbase,ad1base]
    afb=[afbase,af1base]
    eb=[ebase,e1base]
 
    liste=[]
    try:
        counter=0
        for no in range(len(coordinates_author)):   

            ab[counter]['points']=lst[0][counter]
            print(ab[counter])
            liste.append(ab[counter])
            counter=counter+1

        counter=0
        for no in range(len(coordinates_title)):   

            tb[counter]['points']=lst[1][counter]
            print(tb[counter])
            liste.append(tb[counter])
            counter=counter+1    

        counter=0
        for no in range(len(coordinates_journal)):   

            jb[counter]['points']=lst[2][counter]
            print(jb[counter])
            liste.append(jb[counter])
            counter=counter+1
                   
    except:
        print('============================================================================')
        print(number)
        number=number+1
        continue

    import base64
    import json
    # get image data 
    file=names+".jpeg"
    image_data = base64.b64encode(open(file, "rb").read())
    imagedata=str(image_data)[1:].replace("'","")

    x ='''[{
      "version": "4.4.0",  "flags": {},

      "shapes": [
        {
          "label": "author",
          "points": [       [250,269],[250,283],[329,283],[329,269]         ],
          "group_id": null,"shape_type": "polygon", "flags": {}
        },

        {
          "label": "title",
          "points": [        [197,332],[197,350],[383,350],[383,332]      ],
          "group_id": null, "shape_type": "polygon","flags": {}
        },

        {
          "label": "journal",
          "points": [        [202,381],[202, 395],[378, 395],[378, 381]      ],
          "group_id": null,"shape_type": "polygon","flags": {}
        }  ],

      "imagePath": "10939.jpg",
      "imageData": "/9j",
      "imageHeight": 842,
      "imageWidth": 595
    }]'''

    try:
        info=json.loads(x)
        print('User count', len(info))
        print("-------------")      
        # replace shapes value in x JSON 
        info[0]['shapes']=liste
        print(info[0]['shapes'])
        info[0]['imageData']=imagedata
        info[0]['imagePath']=file
        print("-------------------------------------------------------------------------------------------")

        for key in info:
            my_details=key
        name=names+".json"
        with open(name, 'w') as json_file:
            json.dump(my_details, json_file)
    except:
        
        print('============================================================================')
        print(number)
        number=number+1
        continue

    print('============================================================================')
    print(number)
    number=number+1

START
LTPage ---- {'y0': '0', 'y1': '842.04', 'x0': '0', 'x1': '594.96', 'width': '594.96', 'height': '842.04', 'bbox': '[0, 0, 594.96, 842.04]', 'pageid': '1', 'rotate': '0', 'page_index': '0', 'page_label': ''}
LTTextLineHorizontal ---- {'y0': '626.221', 'y1': '639.181', 'x0': '181.73', 'x1': '408.91', 'width': '227.18', 'height': '12.96', 'bbox': '[181.73, 626.221, 408.91, 639.181]', 'word_margin': '0.1'}
LTTextBoxHorizontal ---- {'y0': '626.221', 'y1': '639.181', 'x0': '181.73', 'x1': '408.91', 'width': '227.18', 'height': '12.96', 'bbox': '[181.73, 626.221, 408.91, 639.181]', 'index': '0'}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Michael L. Brodie, Michael Stonebraker 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
END


LTTextLineHorizontal ---- {'y0': '590.917', 'y1': '604.957', 'x0': '76.824', 'x1': '518.83', 'width': '442.006', 'height': '14.04', 'bbox': '[76.824, 590.917, 518.83, 604.957]', 'word_margin': '0.1'}
LTTextBoxHorizontal ---- {'y0': '590.917', 'y1': '604.957', 'x0': '76.824', 'x1':

In [ ]:
# DONE! 